## Quantized model complete evaluation

evaluation
- clean
- noise (random, chessboard, eps = 2-12)
- sgd-uap (untargeted, sgd-uap, whitebox, eps = 2-12)

dataset
- cifar10
- svhn

model_name
- resnet18
- [soft-pruning]
    - resnet18_sfP
    - resnet18_sfP-mixup
    - resnet18_sfP-cutout
- [post-training pruning]
    - resnet18_PX_0.Y
        - X = 2, 3, 4
        - Y = 3, 6, 9

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import json

from utils_attack import uap_batch
from utils_model import *

dir_results = './results/'
dir_uap = './uaps/'

In [2]:
Q_param = 4 #2, 3, 4
model_name = 'resnet20_Q%i' % Q_param

## 1. Evaluate Clean

In [3]:
all_results = []

for dataset in ['cifar10', 'svhn']:
    
    model = get_model(model_name, dataset)
    testloader = get_testdata(dataset)

    top, top_probs, top1acc, top5acc, outputs, labels = evaluate(model, testloader)

    print('Top 1 accuracy', sum(top1acc) / len(labels))
    print('Top 5 accuracy', sum(top5acc) / len(labels))

    dict_entry = {}
    dict_entry['model_name'] = model_name
    dict_entry['dataset'] = dataset
    dict_entry['top1acc'] = sum(top1acc) / len(labels)
    dict_entry['top5acc'] = sum(top5acc) / len(labels)

    for i in range(10):
        dict_entry['label_dist%i' % i] = sum(outputs == i) / len(labels)

    all_results.append(dict_entry)
    pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_clean.csv' % Q_param)

pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_clean.csv' % Q_param)

Loading cifar10/resnet20_Q4.pth
Files already downloaded and verified
 Completed: 0.00%
Top 1 accuracy 0.922
Top 5 accuracy 0.9979
Loading svhn/resnet20_Q4.pth
Using downloaded and verified file: ./data/svhn/test_32x32.mat
 Completed: 0.00%
Top 1 accuracy 0.9498693915181315
Top 5 accuracy 0.994698832206515


In [4]:
def eval_uap(uap):    
    top, top_probs, top1acc, top5acc, outputs, labels = evaluate(model, testloader, uap = uap)

    print('Top 1 accuracy', sum(top1acc) / len(labels))
    print('Top 5 accuracy', sum(top5acc) / len(labels))

    dict_entry = {}
    dict_entry['model_name'] = model_name
    dict_entry['dataset'] = dataset

    dict_entry['atk_name'] = atk_name
    dict_entry['atk_param'] = y_target
    dict_entry['eps'] = eps
    if atk_name[:3] == 'sgd': dict_entry['nb_epoch'] = nb_epoch

    dict_entry['top1acc'] = sum(top1acc) / len(labels)
    dict_entry['top5acc'] = sum(top5acc) / len(labels)
    dict_entry['UER'] = 1 - sum(top1acc) / len(labels)
    print('UER ', dict_entry['UER'])
    print('\n')
    if y_target >= 0: dict_entry['tgt_success'] = sum(outputs == y_target) / len(labels)

    # Output distribution
    for i in range(10):
        dict_entry['label_dist%i' % i] = sum(outputs == i) / len(labels)

    all_results.append(dict_entry)

## 2. Evaluate Noise

In [5]:
all_results = []
y_target = -1

for dataset in ['cifar10', 'svhn']:
    testloader = get_testdata(dataset)
    model = get_model(model_name, dataset)

    # Save directory
    uap_save_path = dir_uap + dataset + '/' + model_name
    if not os.path.exists(uap_save_path):
        os.mkdir(uap_save_path)

    # 1. Chessboard attack
    atk_name = 'chess'
    uap = torch.load(dir_uap + 'chess.pth')
    for eps in np.arange(2, 13, 2):
        eval_uap(uap * eps / 255)

    # 2. Random noise
    atk_name = 'random'
    torch.manual_seed(0)
    uap = torch.sign(torch.rand_like(uap) - 0.5)
    for eps in np.arange(2, 13, 2):
        eval_uap(uap * eps / 255)

    pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_noise.csv' % Q_param)

pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_noise.csv' % Q_param)

Files already downloaded and verified
Loading cifar10/resnet20_Q4.pth
 Completed: 0.00%
Top 1 accuracy 0.8089
Top 5 accuracy 0.9867
UER  0.19110000000000005


 Completed: 0.00%
Top 1 accuracy 0.6168
Top 5 accuracy 0.9425
UER  0.3832


 Completed: 0.00%
Top 1 accuracy 0.4976
Top 5 accuracy 0.902
UER  0.5024


 Completed: 0.00%
Top 1 accuracy 0.417
Top 5 accuracy 0.8638
UER  0.583


 Completed: 0.00%
Top 1 accuracy 0.3522
Top 5 accuracy 0.8271
UER  0.6477999999999999


 Completed: 0.00%
Top 1 accuracy 0.305
Top 5 accuracy 0.7937
UER  0.6950000000000001


 Completed: 0.00%
Top 1 accuracy 0.9179
Top 5 accuracy 0.9973
UER  0.08209999999999995


 Completed: 0.00%
Top 1 accuracy 0.8973
Top 5 accuracy 0.9962
UER  0.10270000000000001


 Completed: 0.00%
Top 1 accuracy 0.862
Top 5 accuracy 0.9912
UER  0.138


 Completed: 0.00%
Top 1 accuracy 0.812
Top 5 accuracy 0.9832
UER  0.18799999999999994


 Completed: 0.00%
Top 1 accuracy 0.7475
Top 5 accuracy 0.9692
UER  0.25249999999999995


 Completed: 

## 3.1. Generate SGD-UAP (untgt, tgt; eps = 2-12)

In [6]:
batch_size = 500
nb_epoch = 10

for dataset in ['cifar10', 'svhn']:
    testloader = get_testdata(dataset, batch_size)
    model = get_model(model_name, dataset, batch_size)

    # Save directory
    uap_save_path = dir_uap + dataset + '/' + model_name
    if not os.path.exists(uap_save_path):
        os.mkdir(uap_save_path)

#     for y_target in range(-1, 10, 1):
    for y_target in [-1]:
    
        if y_target == -1: atk_name = 'sgd-uap'
        else: atk_name = 'sgd-tgt'

        for eps in range(2, 13, 2):
            # Compute UAP
            if y_target == -1:
                save_name = uap_save_path + '/sgd-eps%i.pth' % eps
            else:
                save_name = uap_save_path + '/sgd-tgt%i-eps%i.pth' % (y_target, eps)

            print(save_name)
            if os.path.isfile(save_name):
                print('UAP already generated')
            else:
                if dataset == 'svhn': uap, losses = uap_batch(model, testloader, nb_epoch, eps / 255, beta = 8, y_target = y_target, cut_end = True)
                else: uap, losses = uap_batch(model, testloader, nb_epoch, eps / 255, beta = 8, y_target = y_target)
                torch.save(uap, save_name)

Files already downloaded and verified
Loading cifar10/resnet20_Q4.pth
./uaps/cifar10/resnet20_Q4/sgd-eps2.pth
 Completed: 0.00%
 Completed: 10.00%
 Completed: 20.00%
 Completed: 30.00%
 Completed: 40.00%
 Completed: 50.00%
 Completed: 60.00%
 Completed: 70.00%
 Completed: 80.00%
 Completed: 90.00%
./uaps/cifar10/resnet20_Q4/sgd-eps4.pth
 Completed: 0.00%
 Completed: 10.00%
 Completed: 20.00%
 Completed: 30.00%
 Completed: 40.00%
 Completed: 50.00%
 Completed: 60.00%
 Completed: 70.00%
 Completed: 80.00%
 Completed: 90.00%
./uaps/cifar10/resnet20_Q4/sgd-eps6.pth
 Completed: 0.00%
 Completed: 10.00%
 Completed: 20.00%
 Completed: 30.00%
 Completed: 40.00%
 Completed: 50.00%
 Completed: 60.00%
 Completed: 70.00%
 Completed: 80.00%
 Completed: 90.00%
./uaps/cifar10/resnet20_Q4/sgd-eps8.pth
 Completed: 0.00%
 Completed: 10.00%
 Completed: 20.00%
 Completed: 30.00%
 Completed: 40.00%
 Completed: 50.00%
 Completed: 60.00%
 Completed: 70.00%
 Completed: 80.00%
 Completed: 90.00%
./uaps/cifar10

## 3.2 Evaluate SGD-UAP (untgt, tgt; eps = 2-12)

In [7]:
batch_size = None
nb_epoch = 10

for dataset in ['cifar10', 'svhn']:
    all_results = []
    
    testloader = get_testdata(dataset)

#     for model_name in model_names:
    model = get_model(model_name, dataset)
    model.eval()

    # Load directory
    uap_load_path = dir_uap + dataset + '/' + model_name

#     for y_target in range(-1, 10, 1):
    for y_target in [-1]:

        if y_target == -1: atk_name = 'sgd-uap'
        else: atk_name = 'sgd-tgt'

        for eps in range(2, 13, 2):
            # Compute UAP
            if y_target == -1:
                load_name = uap_load_path + '/sgd-eps%i.pth' % eps
            else:
                load_name = uap_load_path + '/sgd-tgt%i-eps%i.pth' % (y_target, eps)

            print(load_name)                    
            uap = torch.load(load_name)
            eval_uap(uap)

        pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_all-whitebox-%s.csv' % (Q_param, dataset))

    pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_all-whitebox-%s.csv' % (Q_param, dataset))

Files already downloaded and verified
Loading cifar10/resnet20_Q4.pth
./uaps/cifar10/resnet20_Q4/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9035
Top 5 accuracy 0.9961
UER  0.09650000000000003


./uaps/cifar10/resnet20_Q4/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.8442
Top 5 accuracy 0.9897
UER  0.15580000000000005


./uaps/cifar10/resnet20_Q4/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.6699
Top 5 accuracy 0.9551
UER  0.33009999999999995


./uaps/cifar10/resnet20_Q4/sgd-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.5771
Top 5 accuracy 0.9396
UER  0.42290000000000005


./uaps/cifar10/resnet20_Q4/sgd-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.4336
Top 5 accuracy 0.851
UER  0.5664


./uaps/cifar10/resnet20_Q4/sgd-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.3296
Top 5 accuracy 0.7481
UER  0.6704


Using downloaded and verified file: ./data/svhn/test_32x32.mat
Loading svhn/resnet20_Q4.pth
./uaps/svhn/resnet20_Q4/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9406499692685925
